In [ ]:
"""Demo for Loading Solver Design Points.

This script is intended to help with loading a design of solver factors
to run on one or more problems. If more than one problem are used a cross-design of
problems will be run. The design file should be a txt or csv file with headers as
factor names and each row representing an individual design point. Design files
generated by the simopt GUI can also be run using this script. Outputs are printed to
the experiments folder in simopt.
"""

import sys
from pathlib import Path

import pandas as pd

# Append the parent directory (simopt package) to the system path
sys.path.append(str(Path(__file__).resolve().parent.parent))

from simopt.experiment_base import ProblemsSolvers

# run this script in the terminal from the simopt directory
# name of solver that design was created on
solver_name = "RNDSRCH"
# list of problem names for solver design to be run on
# (if more than one version of same problem, repeat name)
problem_names = [
    "SSCONT-1",
    "SAN-1",
]

# name of file containing design points (csv or excel): column headers must exactly
# match names of solver factors w/ each row representing a design point
# (can also use csv's generated by GUI)
design_filename = Path("data_farming_experiments", "RNDSRCH_design.csv")

# list of dictionaries that provide fixed factors for problems when you don't want
# to use the default values, if you want to use all default values use empty
# dictionary, order must match problem names
problem_fixed_factors = [
    {"budget": 2000, "demand_mean": 90.0, "fixed_cost": 25},
    {"budget": 500},
]

# uncomment this version to run w/ only default problem factors
# problem_fixed_factors = [{},{}]

# use this dictionary to change any default solver factors that were not included
# in the design
solver_fixed_factors = {}

# number of macroreplication to run and each solver design point
n_macroreps = 2
# number of post replications to run on each macro replication
n_postreps = 100
# number of normalization postreplications to run at initial solution and optimal
# solution
n_postreps_init_opt = 200

# turn design file into df & retrive dp information
design_table = pd.read_csv(design_filename)

design_factor_names = design_table.columns.tolist()

# remove GUI columns from list if present (ignore if not using design files
# generated by GUI)
design_factor_names.remove("Design #")
design_factor_names.remove("Solver Name")
design_factor_names.remove("Design Type")
design_factor_names.remove("Number Stacks")

dp_list = []  # list of all design points

for _, row in design_table.iterrows():
    dp = {}  # dictionary of current dp
    for factor in design_factor_names:
        dp[factor] = row[factor]
    dp_list.append(dp)

# add fixed solver factors to dps
for fixed_factor in solver_fixed_factors:
    for dp in dp_list:
        dp[fixed_factor] = solver_fixed_factors[fixed_factor]

n_dp = len(dp_list)
solver_names = []
for _ in range(n_dp):
    solver_names.append(solver_name)

experiment = ProblemsSolvers(
    solver_factors=dp_list,
    problem_factors=problem_fixed_factors,
    solver_names=solver_names,
    problem_names=problem_names,
)

experiment.run(n_macroreps)
experiment.post_replicate(n_postreps)
experiment.post_normalize(n_postreps_init_opt)
experiment.record_group_experiment_results()
experiment.log_group_experiment_results()
experiment.report_group_statistics()